# DocumentAssistant Demo

В этом блокноте демонстрируется простой процесс генерации данных с расширенным поиском (RAG)

- загрузка документов (PDF / DOCX / TXT)
- разбивка текста на фрагменты
- семантический поиск с использованием вложений
- генерация ответов с использованием LLM (OpenRouter)

Все данные обрабатываются в памяти.

В качестве базы данных можно использовать SurrealDB Cloud
Её легко подключить:  
await db.connect( 
    "wss://wild-mountain-**workflow id**.aws-euw1.surreal.cloud/rpc",  
    {   
     namespace: process.env.SURREAL_NAMESPACE,   
     database: process.env.SURREAL_DATABASE,  
     auth: {  
        username: String(process.env.SURREAL_USER),  
        password: String(process.env.SURREAL_PASS),  
          },  
    }  
    );  
Можно не запариваться с сериализацией даты, бд сама может с этим разобраться:  

DEFINE FIELD created ON prompts TYPE datetime DEFAULT time::now() READONLY; _Всегда будет ставиться текущее время при создании записи_ 

DEFINE FIELD updated ON prompts TYPE datetime VALUE time::now(); _Value вычисляемое значение, которое будет обновлятся при обновлении записи_  

А также SurrealDB может содержать как вектора так и другие данные в одной базе данных  

In [ ]:
#Импорты

import os
from dotenv import load_dotenv

load_dotenv()

# sanity check
assert os.getenv("OPENROUTER_API_KEY"), "OPENROUTER_API_KEY is not set"

from DocumentAssistant.document_assistant import DocumentAssistant



In [ ]:
#Инициализация агента

assistant = DocumentAssistant(
    chunk_size=500,
    overlap=50,
    top_k=3
)


In [ ]:
#Подготовка документов
documents = [
    "data/A9RD3D4.pdf",
    "data/Polzovatelskoe_soglashenie.pdf",
    "data/University Success.docx",
]
#Документы сохранены локально в data/

In [ ]:
#Индексация документов

assistant.index_documents(documents)

print(f"Indexed {len(assistant.chunks)} text chunks")


In [ ]:
#Первый вопрос
question_1 = "О чем говорится в пользовательском соглашении?"

answer_1 = assistant.answer_query(question_1)
print(answer_1)


In [ ]:
#Второй вопрос
question_2 = "Какие рекомендации даются студентам для успешного обучения?"

answer_2 = assistant.answer_query(question_2)
print(answer_2)


In [ ]:
#Третий вопрос
question_3 = "Какие ограничения или обязанности описаны в документах?"

answer_3 = assistant.answer_query(question_3)
print(answer_3)


In [ ]:
#Сохранение результатов в JSON
import json

results = [
    {"question": question_1, "answer": answer_1},
    {"question": question_2, "answer": answer_2},
    {"question": question_3, "answer": answer_3},
]

with open("results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Results saved to results.json")
